In [2]:
import pickle
from sklearn.base import is_regressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Lees beide datasets in
datasets = {
    "student-mat": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-mat.csv", sep=";"),
    "student-por": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-por.csv", sep=";")
}

target = 'G3'

In [4]:
# Klasse om het model op te slaan en in te laden
class Readwritemodel():
    def __init__(self, model=None, filenaam=None):
        self.filenaam = filenaam
        self.model = model  
    
    def check_input(self, var):
        if isinstance(var, str) and var.endswith(".sav"):
            return True
        else:
            print("Jouw filenaam moet een string zijn en eindigen met .sav")
            return False

    def opslaanmodel(self):
        if self.check_input(self.filenaam):
            pickle.dump(self.model, open(self.filenaam, 'wb'))
            print(f"Jouw model is opgeslagen onder de naam: {self.filenaam}")
        
    def inladenmodel(self):
        if self.check_input(self.filenaam):
            try:
                geladen_model = pickle.load(open(self.filenaam, 'rb'))
                if is_regressor(geladen_model):
                    print("Jouw model is opgehaald en je kan er nu mee voorspellen!")
                    return geladen_model
                else:
                    print("Dit is geen regressie model")
                    return None
            except FileNotFoundError:
                print("Het modelbestand kan niet gevonden worden. Er wordt een nieuw model getraind.")
                return None 

In [5]:
# Modelbeheer
func = Readwritemodel()
func.filenaam = "Regressie.sav"
model = func.inladenmodel()

if model is None:
    print("Ik train een model...")
    model = GradientBoostingRegressor()
    first_dataset = next(iter(datasets.values()))
    first_dummies = pd.get_dummies(first_dataset, drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(first_dummies.drop(columns=[target]), first_dummies[target], test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    func.model = model
    func.opslaanmodel()
else:
    print("Je gebruikt nu een opgeslagen model")

Het modelbestand kan niet gevonden worden. Er wordt een nieuw model getraind.
Ik train een model...
Jouw model is opgeslagen onder de naam: Regressie.sav


In [6]:
# Maak een instantie van de klasse om het model op te slaan of te laden
func = Readwritemodel()
func.filenaam = "Regressie.sav"

# Probeer het model in te laden, anders trainen we een nieuw model
model = func.inladenmodel()

if model is None:
    print("Ik train een model...")
    # Train een nieuw Gradient Boosting model
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)

    # Sla het getrainde model op
    func.model = model
    func.opslaanmodel()
else:
    print("Je gebruikt nu een opgeslagen model")

Jouw model is opgehaald en je kan er nu mee voorspellen!
Je gebruikt nu een opgeslagen model


In [7]:
# Loop door beide datasets en maak voorspellingen
for name, data in datasets.items():
    print(f"Verwerken van dataset: {name}")
    data_dummies = pd.get_dummies(data, drop_first=True)
    X = data_dummies.drop(columns=[target])
    
    # Controleer of de kolommen overeenkomen met de trainingsdata
    missing_cols = set(X_train.columns) - set(X.columns)
    extra_cols = set(X.columns) - set(X_train.columns)
    for col in missing_cols:
        X[col] = 0  # Voeg ontbrekende kolommen toe met 0
    X = X[X_train.columns]  # Houd alleen de relevante kolommen
    
    y_pred = model.predict(X)
    data['Predictions'] = y_pred
    
    output_filename = f"{name}_with_predictions.csv"
    data.to_csv(output_filename, sep=";", index=False)
    print(f"Voorspellingen opgeslagen in {output_filename}")

Verwerken van dataset: student-mat
Voorspellingen opgeslagen in student-mat_with_predictions.csv
Verwerken van dataset: student-por
Voorspellingen opgeslagen in student-por_with_predictions.csv


In [8]:
# Evaluatie op de testset
mse = mean_squared_error(y_test, model.predict(X_test))
r2 = r2_score(y_test, model.predict(X_test))
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

Mean Squared Error (MSE): 4.019045672650393
R-squared (R²): 0.8039972490622082
